In [1]:
#Import Libraries
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
import joblib
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [4]:
#Load Cleaned Data
df = pd.read_csv("../data/processed_data/featured_data.csv")


In [5]:
#Separate Features and Target
# Target column
target = "Weekly_Sales"

# Load feature columns from saved file
with open("../models/feature_columns.pkl", "rb") as f:
    feature_columns = joblib.load(f)

# Input and output
X = df[feature_columns]
y = df[target]


In [7]:
# Load scaler and scale features
scaler = joblib.load("../models/scaler.pkl")
X_scaled = scaler.transform(X)

# Load model
model = load_model("../models/retail_sales_ann_model.h5")

In [8]:
# Predict
y_pred = model.predict(X_scaled).flatten()

# Evaluate
mae = mean_absolute_error(y, y_pred)
mse = mean_squared_error(y, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y, y_pred)

print(f"MAE: {mae:.2f}")
print(f"MSE: {mse:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R² Score: {r2:.4f}")

12968/12968 ━━━━━━━━━━━━━━━━━━━━ 13s 952us/step
MAE: 16000.24
MSE: 771423773.13
RMSE: 27774.52
R² Score: -0.4967


In [12]:
# Save predictions
results = pd.DataFrame({
    "Store": df["Store"],
    "Dept": df["Dept"],
    "Date": df["Date"],
    "Actual_Sales": y,
    "Predicted_Sales": y_pred
})
results.to_csv("../data/predictions/sales_predictions.csv", index=False)